## PR on classifying cookie banner text
- The goal is to see if we could get a Naive-Bayes implementation to work with data gathered from various sources on good and bad cookie banners, manually collected and labeled based on literature from CS 194
- We were able to get it to work, however the results aren't satisfactory and we've hit a wall with how we're better able to pre-process the data to effectively isolate important keywords.

In [22]:
import pandas as pd

df = pd.read_csv('combined_ms.csv')
data = df.to_numpy()

### Processing the text
- Removing punctuation
- Should we take out greetings and the word "cookies"?

In [23]:
import string
import random

word_string = ''
documents = []

for point in data:
    curr_tuple = (point[0].translate(str.maketrans('', '', string.punctuation)).split(), point[1])
    documents.append(curr_tuple)
    word_string = word_string + point[0].translate(str.maketrans('', '', string.punctuation))

random.shuffle(documents)
print(documents[0])

(['We', 'use', 'cookies', 'and', 'other', 'tracking', 'technologies', 'to', 'improve', 'your', 'browsing', 'experience', 'on', 'our', 'site', 'show', 'personalized', 'content', 'and', 'targeted', 'ads', 'analyze', 'site', 'traffic', 'and', 'understand', 'where', 'our', 'audience', 'is', 'coming', 'from', 'To', 'find', 'out', 'more', 'or', 'to', 'optout', 'please', 'read', 'our', 'Cookie', 'Policy', 'In', 'addition', 'please', 'read', 'our', 'Privacy', 'Policy', 'which', 'has', 'also', 'been', 'updated', 'and', 'became', 'effective', 'May', '23rd', '2018', 'By', 'choosing', 'I', 'Accept', 'you', 'consent', 'to', 'our', 'use', 'of', 'cookies', 'and', 'other', 'tracking', 'technologies'], 'GOOD')


### Defining the feature extractor
- Takes five of the words as features

In [24]:
import nltk

# Define the feature extractor

all_words = nltk.FreqDist(w.lower() for w in word_string.split())

word_features = list(all_words)[:5]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

### Training and testing the classifier
- Using NLTK to split the data into testing sets and training sets

In [30]:
# Train Naive Bayes classifier
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[80:], featuresets[:80]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [33]:
# Test the classifier
print(nltk.classify.accuracy(classifier, test_set))

0.75


### Important features
- Noticing they're repeated, doing some reading as to how we can better isolate "relevant" words

In [27]:
# Show the most important features as interpreted by Naive Bayes
classifier.show_most_informative_features(5)

Most Informative Features
           contains(and) = False             BAD : GOOD   =     18.4 : 1.0
           contains(our) = False             BAD : GOOD   =      3.9 : 1.0
           contains(and) = True             GOOD : BAD    =      2.0 : 1.0
            contains(to) = False            GOOD : BAD    =      1.3 : 1.0
           contains(our) = True             GOOD : BAD    =      1.3 : 1.0


## Saving the model

In [ ]:
import pickle5 as pickle

save_classifier = open('calliope.pickle', 'wb')
pickle.dump(classifier, save_classifier) 
save_classifier.close()